In [1]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import ConversationalRetrievalChain

In [2]:
from langchain.document_loaders import TextLoader
loader = TextLoader("data/nio_stumbles.txt")
documents = loader.load()
documents

[Document(page_content='China’s ‘Tesla Killer’ Stumbles as EV Price War Takes Toll\nNIO is among Chinese startups burning more cash to compete in world’s largest electric-vehicle market\n\nSINGAPORE—Electric-car startup NIO NIO -0.36%decrease; red down pointing triangle was dubbed China’s “Tesla killer” when it unveiled a sport-utility vehicle in 2017 that offered a sleek design, large-screen panel and voice-command features—all at half the price of a Model X.\n\nOne of China’s most vaunted EV startups, NIO is now a symbol of the challenges many automakers face amid a cutthroat price war in the world’s largest electric-vehicle market. Its sales have slumped in recent months, prompting the carmaker to slash prices, cut back investment and commit to burning more cash.\n\nChief Executive William Li said this month that New York Stock Exchange-listed NIO has to prudently manage liquidity risks as weak sales in the past two quarters weighed on its operating cash flow. \n\nAn Abu Dhabi gover

In [3]:
# split the document 
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=30)
documents = text_splitter.split_documents(documents)
documents

[Document(page_content='China’s ‘Tesla Killer’ Stumbles as EV Price War Takes Toll\nNIO is among Chinese startups burning more cash to compete in world’s largest electric-vehicle market\n\nSINGAPORE—Electric-car startup NIO NIO -0.36%decrease; red down pointing triangle was dubbed China’s “Tesla killer” when it unveiled a sport-utility vehicle in 2017 that offered a sleek design, large-screen panel and voice-command features—all at half the price of a Model X.\n\nOne of China’s most vaunted EV startups, NIO is now a symbol of the challenges many automakers face amid a cutthroat price war in the world’s largest electric-vehicle market. Its sales have slumped in recent months, prompting the carmaker to slash prices, cut back investment and commit to burning more cash.\n\nChief Executive William Li said this month that New York Stock Exchange-listed NIO has to prudently manage liquidity risks as weak sales in the past two quarters weighed on its operating cash flow.', metadata={'source': 

In [4]:
# create in-memory embeddings for each document 
embeddings = OpenAIEmbeddings()
vectorstore = Chroma.from_documents(documents, embeddings)
vectorstore

In [5]:
# need a memory object so that the qa chain is context aware
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

In [6]:
# An implementation detail:
# Normally we only need to create an embedding for the quesiton itself to fetch for relevant context
# This is not enough for Conversational based retrivel, as the question doesn't contain context
# To solve this, for each question, we rephrase it using the chat history to formulate a standalone
# question. 

qa = ConversationalRetrievalChain.from_llm(OpenAI(temperature=0), vectorstore.as_retriever(), memory=memory)
query = "Who is NIO top competitor?"
result = qa({"question": query})
result["answer"]

BaseConversationalRetrievalChain condensed question: Who is NIO top competitor?


" NIO's top competitor is Tesla."

In [7]:
query = "Why is it a strong competitor?"
result = qa({"question": query})
result["answer"]

BaseConversationalRetrievalChain condensed question:  What makes Tesla a strong competitor to NIO?


' Tesla has been able to maintain strong sales in China, selling more than 200,000 cars to local buyers in the first five months of this year. Tesla also has a well-known brand and has been able to offer competitive prices.'

In [8]:
# Alternatively, we can pass in chat history to QA chain manually 
qa = ConversationalRetrievalChain.from_llm(OpenAI(temperature=0), vectorstore.as_retriever())
tesla_query = "Who is NIO top competitor?"
tesla_result = qa({"question": tesla_query, "chat_history": []})
tesla_result["answer"]

BaseConversationalRetrievalChain condensed question: Who is NIO top competitor?


" NIO's top competitor is Tesla."

In [9]:
# Cont. 
chat_history = [(tesla_query, tesla_result["answer"])]
query = "Why is it a strong competitor?"
result = qa({"question": query, "chat_history": chat_history})
result["answer"]

BaseConversationalRetrievalChain condensed question:  What makes Tesla a strong competitor to NIO?


' Tesla has been able to maintain strong sales in China, selling more than 200,000 cars to local buyers in the first five months of this year. Tesla also has a well-known brand and has been able to offer competitive prices.'